<center>

<span style='color:yellow'>
<center><h1> Task-01</h1> </center>
</span>
<h2> Implement a linear regression model to predict the prices of houses based on their square footage and the number of bedrooms and bathrooms. </h2>



Dataset : - https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data


# -----------------------------------------------------------------------------------

</center>


<center>

# Loading Libreries


</center>

In [140]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor    # For classification tasks
from sklearn.metrics import accuracy_score



<center>

# Importing Data


 </center>

In [83]:
train_data = pd.read_csv("Data/train.csv")
test_data = pd.read_csv("Data/test.csv")
y_test = pd.read_csv("Data/sample_submission.csv")



<center>


# Data Preprocessing

 </center>

In [84]:
y_test

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
...,...,...
1454,2915,167081.220949
1455,2916,164788.778231
1456,2917,219222.423400
1457,2918,184924.279659


In [85]:
test_data=pd.concat((test_data,y_test["SalePrice"]),axis=1)

In [86]:
print(test_data.shape)
print(train_data.shape)

(1459, 81)
(1460, 81)


In [87]:
test_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,169277.052498
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,187758.393989
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,183583.683570
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,179317.477511
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,150730.079977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,167081.220949
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,164788.778231
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,219222.423400
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,184924.279659


In [88]:
test_data.columns == train_data.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [89]:
data=pd.concat((train_data,test_data),axis=0)

In [90]:
data.shape

(2919, 81)

In [91]:
data.iloc[1461]

Id                        1462
MSSubClass                  20
MSZoning                    RL
LotFrontage               81.0
LotArea                  14267
                     ...      
MoSold                       6
YrSold                    2010
SaleType                    WD
SaleCondition           Normal
SalePrice        187758.393989
Name: 1, Length: 81, dtype: object

In [92]:
# Shuffle the rows of the dataframe
data = data.sample(frac=1).reset_index(drop=True)

data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,2067,20,RL,71.0,8838,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2008,WD,Normal,196564.717985
1,2434,20,RL,70.0,7903,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Family,176821.936263
2,2383,20,RL,80.0,9547,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2007,WD,Normal,168991.296358
3,1500,160,FV,24.0,2544,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,150306.307815
4,1382,20,RL,NaN,12925,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2008,WD,Normal,237500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,310,20,RL,90.0,12378,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,11,2006,WD,Normal,360000.000000
2915,1693,80,RL,61.0,9734,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2009,WD,Normal,178630.060560
2916,1772,20,RL,70.0,8750,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdWo,NaN,0,7,2009,WD,Normal,178903.295932
2917,1350,70,RM,50.0,5250,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,122000.000000


In [93]:
# print(data.info())
print(data.shape)

(2919, 81)


In [94]:
new_data = data.select_dtypes(include=['int64', 'float64'])
new_data.info()
new_data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   LotFrontage    2433 non-null   float64
 3   LotArea        2919 non-null   int64  
 4   OverallQual    2919 non-null   int64  
 5   OverallCond    2919 non-null   int64  
 6   YearBuilt      2919 non-null   int64  
 7   YearRemodAdd   2919 non-null   int64  
 8   MasVnrArea     2896 non-null   float64
 9   BsmtFinSF1     2918 non-null   float64
 10  BsmtFinSF2     2918 non-null   float64
 11  BsmtUnfSF      2918 non-null   float64
 12  TotalBsmtSF    2918 non-null   float64
 13  1stFlrSF       2919 non-null   int64  
 14  2ndFlrSF       2919 non-null   int64  
 15  LowQualFinSF   2919 non-null   int64  
 16  GrLivArea      2919 non-null   int64  
 17  BsmtFullBath   2917 non-null   float64
 18  BsmtHalf

(2919, 38)

In [95]:
new_data.isnull().sum()

Id                 0
MSSubClass         0
LotFrontage      486
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
MasVnrArea        23
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
1stFlrSF           0
2ndFlrSF           0
LowQualFinSF       0
GrLivArea          0
BsmtFullBath       2
BsmtHalfBath       2
FullBath           0
HalfBath           0
BedroomAbvGr       0
KitchenAbvGr       0
TotRmsAbvGrd       0
Fireplaces         0
GarageYrBlt      159
GarageCars         1
GarageArea         1
WoodDeckSF         0
OpenPorchSF        0
EnclosedPorch      0
3SsnPorch          0
ScreenPorch        0
PoolArea           0
MiscVal            0
MoSold             0
YrSold             0
SalePrice          0
dtype: int64

In [96]:
new_data.drop(columns=["LotFrontage","GarageYrBlt","MasVnrArea","Id"] , inplace=True)

In [97]:
new_data.shape

(2919, 34)

In [98]:
new_data = new_data.apply(lambda col: col.fillna(col.mean()))

In [99]:
new_data.isnull().sum()

MSSubClass       0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
SalePrice        0
dtype: int64

In [100]:
new_data.shape

(2919, 34)

In [101]:
# data=df_imputed.values
data = new_data.to_numpy()
data

array([[2.00000000e+01, 8.83800000e+03, 5.00000000e+00, ...,
        1.00000000e+01, 2.00800000e+03, 1.96564718e+05],
       [2.00000000e+01, 7.90300000e+03, 5.00000000e+00, ...,
        5.00000000e+00, 2.00700000e+03, 1.76821936e+05],
       [2.00000000e+01, 9.54700000e+03, 7.00000000e+00, ...,
        7.00000000e+00, 2.00700000e+03, 1.68991296e+05],
       ...,
       [2.00000000e+01, 8.75000000e+03, 5.00000000e+00, ...,
        7.00000000e+00, 2.00900000e+03, 1.78903296e+05],
       [7.00000000e+01, 5.25000000e+03, 8.00000000e+00, ...,
        1.20000000e+01, 2.00800000e+03, 1.22000000e+05],
       [2.00000000e+01, 9.90000000e+03, 5.00000000e+00, ...,
        4.00000000e+00, 2.01000000e+03, 1.76954185e+05]])

In [120]:
data.shape

(2919, 34)

In [121]:
X,y = data[:,:-1],data[:,-1]

In [133]:
X

array([[2.000e+01, 8.838e+03, 5.000e+00, ..., 0.000e+00, 1.000e+01,
        2.008e+03],
       [2.000e+01, 7.903e+03, 5.000e+00, ..., 0.000e+00, 5.000e+00,
        2.007e+03],
       [2.000e+01, 9.547e+03, 7.000e+00, ..., 0.000e+00, 7.000e+00,
        2.007e+03],
       ...,
       [2.000e+01, 8.750e+03, 5.000e+00, ..., 0.000e+00, 7.000e+00,
        2.009e+03],
       [7.000e+01, 5.250e+03, 8.000e+00, ..., 0.000e+00, 1.200e+01,
        2.008e+03],
       [2.000e+01, 9.900e+03, 5.000e+00, ..., 0.000e+00, 4.000e+00,
        2.010e+03]])

In [132]:
y.shape

(2919,)

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [123]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2335, 33)
(2335,)
(584, 33)
(584,)


In [124]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [125]:
y_pred = model.predict(X_test)
y_pred.shape

(584,)

In [126]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 1444831300.9290495
R^2 Score: 0.47915488297488895


In [128]:
y_pred_train = model.predict(X_train)
y_pred_train.shape

(2335,)

In [129]:
mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 1792961272.396026
R^2 Score: 0.47584654186281294


In [141]:
model = XGBRegressor()
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [142]:
y_pred = model.predict(X_test)
y_pred.shape

(584,)

In [144]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 1940983576.0966237
R-squared: 0.3002976768389455


In [145]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, r2_score

def test_model(model, X_train=X_train, y_train=y_train):
    cv = KFold(n_splits = 3, shuffle=True, random_state = 45)
    r2 = make_scorer(r2_score)
    r2_val_score = cross_val_score(model, X_train, y_train, cv=cv, scoring = r2)
    score = [r2_val_score.mean()]
    return score

In [146]:
import sklearn.linear_model as linear_model
LR = linear_model.LinearRegression()
test_model(LR)

[0.44083386081853343]

In [147]:
cross_validation = cross_val_score(estimator = LR, X = X_train, y = y_train, cv = 10)
print("Cross validation accuracy of LR model = ", cross_validation)
print("\nCross validation mean accuracy of LR model = ", cross_validation.mean())

Cross validation accuracy of LR model =  [0.35979278 0.42482824 0.50070378 0.47930794 0.47303082 0.46444129
 0.39247453 0.42731452 0.55659232 0.1824022 ]

Cross validation mean accuracy of LR model =  0.42608884382030593


In [148]:
rdg = linear_model.Ridge()
test_model(rdg)

[0.44087785722483286]

In [150]:
lasso = linear_model.Lasso(alpha=1e-3)
test_model(lasso)

[0.4408338638866562]

In [127]:
# with open('Data/column.txt', 'r') as file:
#     columns = file.read().strip().split(',')

In [61]:
# columns

In [62]:
# df_filtered = df[columns]
# print(df_filtered.info())
# df_filtered

In [63]:
# missing_values = df_filtered.isnull().sum()
# print(f"Missing values after imputation:\n{missing_values}")


In [64]:
# missing_values = df_filtered.isnull().sum()
# print(f"Missing values after imputation:\n{missing_values}")


In [65]:
# # data=df_imputed.values
# data = df_filtered.to_numpy()
# data

<center>


# Test and Train Split

 </center>

<center>

# Linear Regression


 </center>